<a href="https://colab.research.google.com/github/Nandhini0707/Nandhini0707/blob/Data-Science-project-using-python/loan_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from google.colab import files

In [37]:
uploaded = files.upload()
df = pd.read_csv('loan_prediction (2).csv')
print(df.head())

Saving loan_prediction (2).csv to loan_prediction (2) (4).csv
   Unnamed: 0   Loan_ID Gender Married Dependents     Education Self_Employed  \
0           0  LP001002   Male      No          0      Graduate            No   
1           1  LP001003   Male     Yes          1      Graduate            No   
2           2  LP001005   Male     Yes          0      Graduate           Yes   
3           3  LP001006   Male     Yes          0  Not Graduate            No   
4           4  LP001008   Male      No          0      Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0         NaN             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   

   Credit_History Proper

In [ ]:
df = df.drop('Loan_ID',axis=1)

In [54]:
df.isnull().sum()

Unnamed: 0            0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [55]:
print(df.describe())

       Unnamed: 0  ApplicantIncome  CoapplicantIncome  LoanAmount  \
count  614.000000       614.000000         614.000000  592.000000   
mean   306.500000      5403.459283        1621.245798  146.412162   
std    177.390811      6109.041673        2926.248369   85.587325   
min      0.000000       150.000000           0.000000    9.000000   
25%    153.250000      2877.500000           0.000000  100.000000   
50%    306.500000      3812.500000        1188.500000  128.000000   
75%    459.750000      5795.000000        2297.250000  168.000000   
max    613.000000     81000.000000       41667.000000  700.000000   

       Loan_Amount_Term  Credit_History  
count         600.00000      564.000000  
mean          342.00000        0.842199  
std            65.12041        0.364878  
min            12.00000        0.000000  
25%           360.00000        1.000000  
50%           360.00000        1.000000  
75%           360.00000        1.000000  
max           480.00000        1.000000  


fill missing values in categorial columns with mode

In [56]:
df['Gender'].fillna(df['Gender'].mode()[0], inplace=True)
df['Married'].fillna(df['Married'].mode()[0], inplace=True)
df['Dependents'].fillna(df['Dependents'].mode()[0], inplace=True)
df['Self_Employed'].fillna(df['Self_Employed'].mode()[0], inplace=True)

fill the missing values of
      1)loan amount column with the median value
      2)loan amount term column with the mode value of the column
      3)credit history column with the mode value

In [57]:
df['LoanAmount'].fillna(df['LoanAmount'].median(), inplace=True)

df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mode()[0], inplace=True)

df['Credit_History'].fillna(df['Credit_History'].mode()[0], inplace=True)

EDA

In [58]:
import plotly.express as px

loan_status_count = df['Loan_Status'].value_counts()
fig_loan_status = px.pie(loan_status_count,
                         names=loan_status_count.index,
                         title='Loan Approval Status')
fig_loan_status.show()

In [59]:
gender_count = df['Gender'].value_counts()
fig_gender = px.bar (gender_count,
                    x=gender_count.index,
                    y=gender_count.values,


SyntaxError: ignored

In [60]:
married_count = df['Married'].value_counts()
fig_married = px.bar(married_count,
                     x=married_count.index,
                     y=married_count.values,
                     title='Marital Status Distribution')
fig_married.show()

In [61]:
education_count = df['Education'].value_counts()
fig_education = px.bar(education_count,
                       x=education_count.index,
                       y=education_count.values,
                       title='Education Distribution')
fig_education.show()

In [62]:
fig_applicant_income = px.histogram(df, x='ApplicantIncome',
                                    title='Applicant Income Distribution')
fig_applicant_income.show()

relationship between the income of the loan applicant & the loan status:

In [63]:
fig_income = px.box(df, x='Loan_Status',
                    y='ApplicantIncome',
                    color="Loan_Status",
                    title='Loan_Status vs ApplicantIncome')
fig_income.show()

In [64]:
# Calculate the IQR
Q1 = df['ApplicantIncome'].quantile(0.25)
Q3 = df['ApplicantIncome'].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove outliers
df = df[(df['ApplicantIncome'] >= lower_bound) & (df['ApplicantIncome'] <= upper_bound)]

In [65]:
fig_coapplicant_income = px.box(df,
                                x='Loan_Status',
                                y='CoapplicantIncome',
                                color="Loan_Status",
                                title='Loan_Status vs CoapplicantIncome')
fig_coapplicant_income.show()

In [69]:
fig_loan_amount = px.box(df, x='Loan_Status',
                         y='LoanAmount',
                         color="Loan_Status",
                         title='Loan_Status vs LoanAmount')
fig_loan_amount.show()

In [70]:
fig_credit_history = px.histogram(df, x='Credit_History', color='Loan_Status',
                                  barmode='group',
                                  title='Loan_Status vs Credit_His')
fig_credit_history.show()

In [71]:
fig_property_area = px.histogram(df, x='Property_Area', color='Loan_Status',
                                 barmode='group',
                                title='Loan_Status vs Property_Area')
fig_property_area.show()

Data Preparation and Training Loan Approval Prediction Model

In [72]:
# Convert categorical columns to numerical using one-hot encoding
cat_cols = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']
df = pd.get_dummies(df, columns=cat_cols)

# Split the dataset into features (X) and target (y)
X = df.drop('Loan_Status', axis=1)
y = df['Loan_Status']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the numerical columns using StandardScaler
scaler = StandardScaler()
numerical_cols = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

from sklearn.svm import SVC
model = SVC(random_state=42)
model.fit(X_train, y_train)


SVC(random_state=42)

In [73]:
y_pred = model.predict(X_test)
print(y_pred)

['Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y']


In [76]:
# Convert X_test to a DataFrame
X_test_df = pd.DataFrame(X_test, columns=X_test.columns)

# Add the predicted values to X_test_df
X_test_df['Loan_Status_Predicted'] = y_pred
print(X_test_df.head(30))

     Unnamed: 0  ApplicantIncome  CoapplicantIncome  LoanAmount  \
585         585         0.045014           0.374594    0.619997   
569         569        -0.531280           0.085836   -0.541877   
255         255        -0.574102          -0.550912    2.038168   
570         570        -0.401782          -0.011034    0.859207   
529         529         1.334840          -0.550912   -0.097631   
346         346        -0.347093           0.445550    0.278270   
269         269        -0.680900          -0.069649   -0.781087   
541         541        -1.061657          -0.550912   -0.627309   
421         421        -0.761385          -0.550912   -0.951951   
242         242        -0.634982           1.207859    0.329529   
86           86        -0.445120           0.066092   -0.627309   
163         163        -0.014834          -0.104510    0.380788   
279         279        -0.049401          -0.550912   -0.200149   
215         215        -0.178384          -0.247655   -0.61022